## Extract Infomration From ACC URL Opening 

Example URL , make sure you opened the 3d model in the browser before copying the URL

In [1]:

url = "https://acc.autodesk.com/docs/files/projects/ec0f8261-aeca-4ab9-a1a5-5845f952b17d?folderUrn=urn%3Aadsk.wipprod%3Afs.folder%3Aco.SQBGbt7LTWywCVcHfqSb1Q&entityId=urn%3Aadsk.wipprod%3Adm.lineage%3AoxAqhUL0SISdkPcNLzehag&viewModel=detail&moduleId=folders&viewableGuid=5d41dda7-eea1-eff5-77dd-ee1aa81fc3a8"

In [2]:
import requests
from urllib.parse import urlparse

def get_info_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        parsed_url = urlparse(url)
        
        # Extract project ID from URL path
        path_segments = parsed_url.path.split('/')
        project_id = path_segments[4] if len(path_segments) >= 4 else ''
        # Extract query parameters
        query_params = dict(q.split("=") for q in parsed_url.query.split("&"))
        folder_urn = query_params.get('folderUrn', '')
        entity_id = query_params.get('entityId', '')
        viewable_guid = query_params.get('viewableGuid', '')
        
        return {
            'ProjectId': "b."+project_id,
            'folderUrn': folder_urn,
            'entityId': entity_id,
            'viewableGuid': viewable_guid
        }
    except Exception as e:
        print("Error occurred:", e)
        return None
info = get_info_from_url(url)
if info:
    for key, value in info.items():
        print(f"{key}: {value}")


ProjectId: b.ec0f8261-aeca-4ab9-a1a5-5845f952b17d
folderUrn: urn%3Aadsk.wipprod%3Afs.folder%3Aco.SQBGbt7LTWywCVcHfqSb1Q
entityId: urn%3Aadsk.wipprod%3Adm.lineage%3AoxAqhUL0SISdkPcNLzehag
viewableGuid: 5d41dda7-eea1-eff5-77dd-ee1aa81fc3a8


### Get Derivative URN

In [3]:
# convert entityId to derivative urn by base64encode
import base64
version = 2
item_version = info['entityId']+ f"?version={version}"
urn = base64.b64encode(item_version.encode()).decode()
# change character "/" to "_
urn = urn.replace("/", "_")
print(f"Derivative URN Of Version {version}: {urn}")

Derivative URN Of Version 2: dXJuJTNBYWRzay53aXBwcm9kJTNBZG0ubGluZWFnZSUzQW94QXFoVUwwU0lTZGtQY05MemVoYWc_dmVyc2lvbj0y


This URN not it correct urn we need : https://stackoverflow.com/questions/78118458/convert-urn-from-item-version-have-problem/78120646#78120646

## This is next step to get derivative URN


In [4]:
from aps_toolkit import BIM360
from aps_toolkit import Auth

In [11]:
token = Auth().auth2leg()
bim360 = BIM360(token)

In [6]:
df = bim360.batch_report_item_versions(info['ProjectId'], info['entityId'])
df

,item_id,version,derivative_urn,last_modified_time
0,urn%3Aadsk.wipprod%3Adm.lineage%3AoxAqhUL0SISd...,2,dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLm94QXFoVU...,2024-03-06T13:59:04.0000000Z


In [7]:
print(df.iloc[0]['derivative_urn'])

dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLm94QXFoVUwwU0lTZGtQY05MemVoYWc_dmVyc2lvbj0y


In [8]:
urn = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLm94QXFoVUwwU0lTZGtQY05MemVoYWc_dmVyc2lvbj0y"

In [9]:
from aps_toolkit import PropDbReaderRevit

In [12]:
prop_reader = PropDbReaderRevit(urn, token)

In [15]:
df = prop_reader.get_data_by_categories(['Walls', 'Doors', 'Windows'])
df.head()

,dbId,external_id,ElementId,Name,Category,CategoryId,Level,Structural,Structural Usage,Location Line,...,Stile Width - Latch,Trim Projection Ext,Trim Projection Int,Trim Width,Jamb Width,Frame Projection,panel,frame,trim projection,Panel Material
0,3473,5e0bc1aa-5715-46e1-a969-6e93a8d1c547-000db7ee,899054,Basic Wall [899054],Revit Walls,-2000011,2.0,0.0,Non-bearing,Core Centerline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3480,5e0bc1aa-5715-46e1-a969-6e93a8d1c547-000db87d,899197,Basic Wall [899197],Revit Walls,-2000011,2.0,0.0,Non-bearing,Core Centerline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3481,5e0bc1aa-5715-46e1-a969-6e93a8d1c547-000dba1e,899614,Basic Wall [899614],Revit Walls,-2000011,2.0,0.0,Non-bearing,Core Centerline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3489,62d50f22-e17b-44bb-a9ae-91e637a76406-000dd866,907366,Basic Wall [907366],Revit Walls,-2000011,2.0,0.0,Non-bearing,Core Centerline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3497,62d50f22-e17b-44bb-a9ae-91e637a76406-000dd98f,907663,Basic Wall [907663],Revit Walls,-2000011,2.0,0.0,Non-bearing,Core Centerline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
